In [1]:
import urllib.request, json 
import pandas as pd
from pandas.io.json import json_normalize
import requests

resp = requests.get('https://www.metro.net/wp-json/wp/v2/line-override/', timeout=10,headers={'Content-Type': 'application/json'})
df= pd.read_json(resp.text)

cleaned_df = df.drop(['_links','yoast_head','yoast_head_json'],axis=1)
cleaned_df.to_csv('resp.csv',index=False)